# KLOOK_주문처리_자동화
### 프로세스 :
- 1. Gmail API 연동 후 주문 메일 가져오기
- 2. 가져온 주문 정보를 KLOOK 주문 사이트와 연동
- 3. 연동된 데이터를 전처리 및 암호화 후 API주소에 송신
- 4. 송신된 데이터는 DB에서 확인 가능
- 5. DB에 들어간 데이터를 확인 후 생성한 바우처 번호를 다시 KLOOK 측에 전송 및 등록


In [1]:
import datetime as dt, pandas as pd, datetime as dt
import os, time, pymysql, re, requests, pickle, os.path, base64, email, hashlib, crypto, sys
from openpyxl import load_workbook
from googleapiclient.discovery import build

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from __future__ import print_function
from base64 import b64encode
sys.modules['Crypto'] = crypto
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES


import pymysql

def ts(num : int):
    time.sleep(num)

def live_db_conn():
    conn = pymysql.connect(host='host', user='user', password='password',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "database")
    return conn

def data_encoder(text):
    if len(text)>0:
        message = email.message_from_string(str(base64.urlsafe_b64decode(text), 'utf-8'))
    return message

def change_time(x):
    x = x.lower().replace(".",'').replace(" ",'').replace("::",':').replace("點",'')
    if 'pm' in x.lower() :
        if len(re.sub(r'[^0-9]','',x)) == 3 :
            x = '{}:{}'.format(12+int(x[0]),x[1:]).replace("::",':')
        elif len(x.lower().replace("pm",'')) == 1 :
            x = "{}:00".format(int(x.lower().replace("pm",'')) + 12)
    elif len(x)== len(re.sub(r'[^0-9]','',x)) == 4 :
        x = "{}:{}".format(x[:2],x[2:]) 
    elif len(re.sub(r'[^0-9]','',x)) == 3 :
        x = "{}{}".format(int(x[0])+12, x[1:]) 
    return x

def get_date(string):
    y, m = string.strip()[12:].split()[0], dt.datetime.strptime(string.strip()[8:].split()[0],'%b').month#.tm_mon
    d,t = string.strip()[5:].split()[0]  , string.strip()[17:25]
    bt = "{}:{}:{} {}".format(y,m,d,t)
    result = dt.datetime.strptime(bt , "%Y:%m:%d %H:%M:%S") + dt.timedelta(hours=16)
    return result.strftime("%Y-%m-%d %H:%M:%S")

def encryptString(str, secret_key, secret_iv):
    key = hashlib.sha256(secret_key.encode('utf-8')).hexdigest()[:32].encode('utf-8')
    iv = hashlib.sha256(secret_iv.encode('utf-8')).hexdigest()[:16].encode('utf-8')
    data = pad(str.encode(), AES.block_size)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return b64encode(b64encode(cipher.encrypt(data)).decode("utf-8").encode()).decode("utf-8").replace("=","")

        
def Order_Automation(N_days):
    
    start_time = time.localtime().tm_mday
    while True:

        try:
            
            # N_days일 후, 오전 6시 이후면 3시간 뒤면 9시 넘으니까 코드 중지. 출근해서 직접 돌리면 됨.
            if N_days > 0 and time.localtime().tm_mday - start_time == N_days and time.localtime().tm_hour >= 8:
                print('1시간 뒤 9시를 넘으므로 코드를 중지합니다.\n현재 시간: ', dt.datetime.now())
                break
            
            # Google Mail API 사용 토큰, Google Mail API 사용 용도, REDTABLE 예약 및 취소 API 
            creds, SCOPES, URL_LIVE = None, ['https://www.googleapis.com/auth/gmail.readonly'], 'https://apiv1.redtable.global/ota/order-create'
            # REDTABLE 암호화 정보
            secret_key, secret_iv = "******", "*******"
            
            # Token을 통해 Credentials 생성
            if os.path.exists('token.pickle'):

                with open('token.pickle', 'rb') as token:

                    creds = pickle.load(token)

            # Token이 만료되거나 문제가 있는 경우        
            if not creds or not creds.valid:
                
                # Refresh
                if creds and creds.expired and creds.refresh_token:
                    
                    creds.refresh(Request())
                    
                # 해당 경로를 통해 Credentials 생성
                else:

                    flow = InstalledAppFlow.from_client_secrets_file('client_secret_992825797470-qefoh2u5gru8u2ng784de8rbf7lum0mf.apps.googleusercontent.com.json', SCOPES)
                    creds = flow.run_local_server(port=0)
                
                # 정상적인 token, credentials 생성
                with open('token.pickle', 'wb') as token:
                    
                    pickle.dump(creds, token)


            conn = live_db_conn()
            curs = conn.cursor()

            start= time.time()
        
            # Klook 계약 매장 
            sql = """SELECT OS.store_id, S.name ,OS.store_id_ext, OS.memo, SC.order_confirm
            FROM ota_store OS, store S ,store_contract SC 
            WHERE OS.ota_id = 3 AND OS.store_id = S.id AND SC.store_id = S.id"""
            curs.execute(sql)
            otas = pd.DataFrame(curs.fetchall())

            # Klook 계약 매장의 상품
            sql = """SELECT OP.memo, OP.product_id, OP.product_id_ext, price FROM ota_product OP WHERE OP.ota_id = 3"""
            curs.execute(sql)
            otap = pd.DataFrame(curs.fetchall())

            # 예약 건 확인
            sql = """SELECT O.store_id, O.admin_id, O.manager_id , O.order_at , case when O.confirm_at is null then 0 else 1 end 'confirm_at', O.ota_order_no , O.manager_memo, O.name, O.status, SC.order_confirm, O.need_confirm
            FROM `order` O
            LEFT JOIN store_contract SC ON SC.store_id = O.store_id
            WHERE O.ota_id = 3"""
            curs.execute(sql)
            order = pd.DataFrame(curs.fetchall())

            print('============================================')
            print('DB 불러오기 완료!')
            
            # BUILD - Google Mail API  
            service = build('gmail', 'v1', credentials=creds)
        
            print('--------------------------------------------')
            print('<Part 1. 취소 메일>')
            print('취소 건 확인을 시작합니다.')
            
            try:

                cancel_messages = service.users().messages().list(userId='me', q='subject:Klook Order Canceled -Reminder').execute()['messages']
                print('정상적으로 취소 메일을 불러왔습니다.')

            except Exception as cancel_fail:
                
                print('취소 메일 불러오기 로딩 실패: ', cancel_fail)
                # print('요청 횟수 초과로 예약취소 메일 접근이 제한되었습니다. 권장 시간 이후에 진행해주세요.')
            
            for message_id in cancel_messages:

                message = service.users().messages().get(userId='me', id= message_id['id']).execute()
                order_no = ''
                cancel_cnt = 0

                try : 
                    # 취소 메일 속에서 주문 번호를 확인
                    if 'order' in message['snippet'].lower() and 'cancel' in message['snippet'].lower():

                        if 'Booking reference ID: ' in message['snippet']:

                            order_no = message['snippet'].split("Booking reference ID: ")[1].split()[0].strip()

                        elif 'parts' in message['payload'] and 'Booking reference ID: ' in str(data_encoder(message['payload']['parts'][0]['body']['data'])):

                            order_no = str(data_encoder(message['payload']['parts'][0]['body']['data'])).split('Booking reference ID: ')[1].split("\n")[0]

                        elif 'parts' not in message['payload']:

                            order_no = pd.read_html(str(data_encoder(message['payload']['body']['data'])))[5][1][0].split("Booking reference ID: ")[1].strip().split()[0]

                        else:
                            
                            text = "취소된 주문번호를 찾을 수 없습니다."
                            print(text)

                        # 해당 주문 번호의 상태 값을 취소로 변경
                        if len(order[(order['ota_order_no']== order_no) & (order['status'] !='cancel')]) > 0:

                            sql = """UPDATE `order` O
                            SET O.status = 'cancel', O.cancel_at = current_date, O.cancel_price = O.order_price
                            WHERE O.status != 'cancel' AND O.ota_order_no = '{}'""".format(order_no)
                            curs.execute(sql)

                            print('{} - 취소 반영 완료 '.format(order_no))
                            cancel_cnt += 1

                        else:
                            
                            continue

                    else:
                        
                        continue

                except:

                    text = "취소 갱신 에러: 확인필요"
                    print(text)

            print('취소 건 확인 완료. 취소 건은 총', cancel_cnt,'건입니다.')
            print('--------------------------------------------')

            print('<Part 2. 예약 메일>')
            print('예약 건 확인을 시작합니다.')
            try:
                
                messages = service.users().messages().list(userId='me', q='subject:Klook has received an order -Dear team').execute()['messages']
                print('정상적으로 예약 메일을 불러왔습니다.')

            except Exception as limit_exceeded_2:
                
                print(limit_exceeded_2)
                
           
            # 불필요한 단어 리스트 (후에 제거)
            re_word = "Booking reference ID:|Date Request:|Time Request:|Lead participant:|Country/region of passport:|Participant:|Activity URL:|Number of Diners:"

            # 기본 데이터프레임 
            df = pd.DataFrame(columns= ['store_id', 'products', 'total_price', 'pay_method', 'currency','booking_time', 'reserve_day', 'reserve_time', 'name', 'email', 'phone','memo', 'person', 'ota_code', 'ota_key', 'ota_sns_type', 'ota_sns_id','ota_country', 'ota_order_no', 'confirm_at', 'order_confirm'])

            # 바우처 번호를 부야해주기 위한 틀 생성
            oo = order[(order['ota_order_no'].notnull()) & (order['manager_memo'].notnull())]['manager_memo'].apply(lambda x : str(x).split(":")[1].split(",")[0].strip().replace('요청사항',''))
            
            
            # 당일 예약 건이 이미 존재하면, 해당 건의 다음 번호를 부여
            if len(oo[oo.str[:6] == dt.datetime.now().strftime("%y%m%d")].apply(lambda x : int(x[-4:]))) > 0:    
                
                voucher_num = max(oo[oo.str[:6] == dt.datetime.now().strftime("%y%m%d")].apply(lambda x : int(x[-4:]))) + 1

            # 당일 첫 예약 건인 경우, 1 부여
            else:
                
                voucher_num = 1

            # 예약 내용 확인
            
            for message_id in messages:

                message = service.users().messages().get(userId='me', id= message_id['id']).execute()

                if str(message['payload']['body']) != "{'size': 0}":

                    contents = pd.read_html(str(data_encoder(message['payload']['body']['data'])))[6][0]
                    
                else:

                    contents = pd.read_html(str(data_encoder(message['payload']['parts'][1]['body']['data'])))[6][0]
                    

                try:
                    sie = contents[6].split("/")[-1]
                    pn = message['snippet'][51:].split(" - ")[1].split("Please")[0][:-2].replace("amp;",'').strip()
                    
                except:

                    contents = pd.read_html(str(data_encoder(message['payload']['body']['data'])))[9][0]
                    sie = contents[6].split('/')[-1]
                    pn = message['snippet'][51:].split(" - ")[1]
             
                
                tmp = pd.DataFrame(map(lambda x : re.sub(re_word ,"", x).strip(), contents[:6].values)).transpose()
                tmp.columns = ['ota_order_no','reserve_day','reserve_time','name','ota_country','person']
                tmp['memo'] = 'X'
                
                if tmp['ota_order_no'][0].strip() in list(map(lambda x : x.strip() , order[(~order['ota_order_no'].isnull())]['ota_order_no'])):
                
                    continue

                for content in contents[5:]: 
                    
                    if 'Preferred' in content:
                        
                        tmp['reserve_time'] = content.split('  ')[1]

                  
                            
                    elif 'Number of Diners:' in content:

                        tmp['person'] = str(re.sub(re_word,'', content)).strip()
                    
                    elif 'No. of participants:' in content:
        
                        tmp['person'] = content.split(':')[1].strip()
                        
                    else:

                        tmp['memo'] = content

                tmp['booking_time'] = get_date(message['payload']['headers'][3]['value'].split(";")[1])[:-3]
                
                # Product Name + 대,중,소
                if (',' not in tmp['person'][0]) and ('Medium' in tmp['person'][0]):
                    pn = pn + '-Medium'

                elif (',' not in tmp['person'][0]) and ('Large' in tmp['person'][0]):
                    pn = pn + '-Large'

                elif (',' not in tmp['person'][0]) and ('Small' in tmp['person'][0]):

                    pn = pn + '-Small'


                if sie in dict(otas[['store_id_ext','store_id']].values):

                    tmp['store_id'] = dict(otas[['store_id_ext','store_id']].values).get(sie)

                    if (',' not in tmp['person'][0]) and (tmp['store_id'][0] == 35465): # 롯데호텔
                        
                        if 'Adult' in tmp['person'][0]:

                            pn = pn + '-Adult'

                        elif 'Child' in tmp['person'][0]:

                            pn = pn + '-Child'

                    elif (',' not in tmp['person'][0]) and (tmp['store_id'][0] == 10754):
                        
                            
                        if 'Adult' in tmp['person'][0]:

                            pn = pn + '-Adult'

                        elif 'Child' in tmp['person'][0]:
                            
                            pn = pn + '-Child(Elementary school student)'

                        elif 'Infant' in tmp['person'][0]:

                            pn = pn + '-Infant(Preschooler)'


                    tmp['order_confirm'] = dict(otas[['store_id_ext','order_confirm']].values).get(sie)
                    tmp['confirm_at'] = dict(order[['store_id','confirm_at']].values).get(tmp['store_id'][0])


                if ',' in tmp['person'][0]: # PERSON TYPE 여러명
                
                    person_type = tmp['person'][0].split(',') # 2 X adult, 1 X infant

                    if person_type == '10' or person_type == '20' or person_type == '30' or person_type == '50':

                        continue

                    else:

                        for i, type_n in enumerate(person_type):

                            size_type = type_n.split('x')[1].strip()
                            quantity = type_n.split('x')[0].strip()
                            pn_sized = pn + '-' + size_type

                            if i == 0:
                            

                                if pn_sized in dict(otap[['memo','product_id']].values):

                                    tmp['products'] = [[{"product_id" : str(dict(otap[['memo','product_id']].values).get(pn_sized)) , 'product_price' : int(dict(otap[['memo','price']].values).get(pn_sized)), 'product_quantity' : int(contents[5].split(": ")[1].split(" x ")[0].strip())}]]
                                    

                                else:
                                    
                                    text = 'DB에 해당 Product가 존재하지않습니다.: {}'.format(pn_sized)
                                    print(text)
                            else:
                                
                                temp_list = tmp['products'].values.tolist()[0]
                                if pn_sized in dict(otap[['memo','product_id']].values):
                                    
                                    temp_list.append({"product_id" : str(dict(otap[['memo','product_id']].values).get(pn_sized)) , 'product_price' : int(dict(otap[['memo','price']].values).get(pn_sized)), 'product_quantity' : int(quantity)})
                            
                                    tmp['products'] = [temp_list]
                                    
                                else:

                                    text = 'DB에 해당 Product가 존재하지않습니다.: {}'.format(pn_sized)
                                    print(text)

                    tmp['total_price'] = 0
                    product_list = tmp['products'][0]
                    voucher_num += 1


                    for i in range(len(product_list)):

                        tmp['total_price'] += product_list[i]['product_quantity'] * product_list[i]['product_price']

                    if  len(str(voucher_num)) == 1:

                            tmp['memo'] = "바우처 번호 : {}-REDTABLE-000{} , 요청사항 : {}".format(dt.datetime.now().strftime("%y%m%d"), voucher_num, tmp['memo'][0])
                    

                    else:

                        tmp['memo'] = "바우처 번호 : {}-REDTABLE-00{} , 요청사항 : {}".format(dt.datetime.now().strftime("%y%m%d"), voucher_num, tmp['memo'][0])

                else:


                    if pn in dict(otap[['memo','product_id']].values):

                        tmp['products'] = [{"product_id" : str(dict(otap[['memo','product_id']].values).get(pn)) , 'product_price' : int(dict(otap[['memo','price']].values).get(pn)), 'product_quantity' : int(contents[5].split(": ")[1].split(" x ")[0].strip())}]
                        
                    
                        voucher_num += 1

                        if  len(str(voucher_num)) == 1:

                            tmp['memo'] = "바우처 번호 : {}-REDTABLE-000{} , 요청사항 : {}".format(dt.datetime.now().strftime("%y%m%d"), voucher_num, tmp['memo'][0])
                    

                        else:

                            tmp['memo'] = "바우처 번호 : {}-REDTABLE-00{} , 요청사항 : {}".format(dt.datetime.now().strftime("%y%m%d"), voucher_num, tmp['memo'][0])
                
                        tmp['total_price'] = tmp['products'].apply(lambda x : str(x['product_quantity'] * x['product_price']) if type(x)!= float else '')

                    else:
                        
                        
                        text = 'DB에 해당 Product가 존재하지않습니다.: {} - {}'.format(pn, tmp['ota_order_no'][0])
                        print(text)
                    
                    if len(tmp['person'][0]) >= 15:
                        
                        tmp['person'] = '-'

                
                df = pd.concat([df,tmp])
                
            
            if len(df) != 0:

                df['reserve_time'] = df['reserve_time'].apply(lambda x : change_time(x))
                df['person'] = df['person'].apply(lambda x : x.replace(" x ",' ').replace("adults ","adults, "))
                df['name'] = df['name'].apply(lambda x : x.replace("()",'').replace(")",') '))
                # df['total_price'] = df['products'].apply(lambda x : str(x['product_quantity'] * x['product_price']) if type(x)!= float else '')
                df['products'] = df['products'].apply(lambda x : str([x]) if type(x) == dict else x)
                
                df[['pay_method','currency','ota_code','ota_key']] = ['card','KRW', 'klook', 'L2w5SGIvUFcrbzZzc3huSnVZSjhnZz09']
                
                df[['store_id','total_price','person']] = df[['store_id','total_price','person']].astype(str)
                
                df['memo'] = df['memo'].apply(lambda x : x.replace("-0",'-') if len(x.split("-REDTABLE-")[1].split()[0]) == 5 else x).str.replace(u"\xa0",' ').replace('don"t',"don't")
                
                df['reserve_time'] = df['reserve_time'].apply(lambda x : x if x != 'NA' else '00:00')
                
                df['ota_order_no'] = df['ota_order_no'].str.strip()
                
                df.fillna(inplace = True, value='')

                df['reserve_time'] = df['reserve_time'].str.replace("am",'')
            
                
                reserve_cnt = 0
                for i in range(len(df)):
                    
                    sql = """SELECT O.store_id, O.admin_id, O.manager_id , O.order_at , case when O.confirm_at is null then 0 else 1 end 'confirm_at', O.ota_order_no , O.manager_memo, O.name, O.status, SC.order_confirm, O.need_confirm
                    FROM `order` O
                    LEFT JOIN store_contract SC ON SC.store_id = O.store_id
                    WHERE O.ota_id = 3"""
                    curs.execute(sql)
                    order = pd.DataFrame(curs.fetchall())
                    
                    if df.iloc[i]['ota_order_no'].strip() in list(map(lambda x : x.strip() , order[(~order['ota_order_no'].isnull())]['ota_order_no'])):
                        
                        continue

                    elif df.iloc[i]['ota_order_no'].strip() not in list(map(lambda x : x.strip() , order[(~order['ota_order_no'].isnull())]['ota_order_no'])):
                    
                        print('주문번호 {}을 DB에 등록합니다'. format(df.iloc[i]['ota_order_no']))

                        PARAMS = str(df.iloc[i,:-2].to_dict())
                        data = PARAMS.replace("'",'"').replace(" && ","&&").replace('"[','[').replace(']"',']').replace("：",':').replace(" : ",':').replace("  ",' ').replace("Special requirements:",'').replace("{store",'{"store').replace('pm"','"').replace("::",":").replace('"d ','`d')
                        r = encryptString(data, secret_key, secret_iv).replace("=",'')
            
                        req = requests.post(url = URL_LIVE, data = {'data' : r})

                        if req.status_code == 200:

                            print('{} - DB 등록 완료'.format(df.iloc[i]['ota_order_no']))
                            reserve_cnt += 1

                        else: 

                            text = '{} - DB 등록 실패'.format(df.iloc[i]['ota_order_no'])
                            
                            
                            print(text)
                            print('등록 실패 내용')
                            print(req.content)
                            
                print('모든 건이 예약이 완료되었습니다.')
                print('예약 건 확인 완료. 예약 건은 총', reserve_cnt,'건입니다.')
                print('--------------------------------------------')
                sql = """
                UPDATE `order` O
                SET O.manager_memo = SUBSTRING_INDEX(O.memo, ",",1), O.memo = REPLACE(O.memo, CONCAT(SUBSTRING_INDEX(O.memo, ",",1),", "), "")
                WHERE LEFT(O.memo , 7) = "바우처 번호:" AND O.manager_memo IS NULL
                """
                curs.execute(sql)
                curs.close()
                conn.close()

                
            else:

                print('추가 등록된 예약 건이 없습니다.')
            
            print("소요 시간 : {}".format(time.time() - start))
            print("완료 시간 :",dt.datetime.now())
            print('--------------------------------------------')


        except Exception as e1:

            if N_days == 0:

                break

            try:

                curs.close()
                conn.close()

            except Exception as e2:

                print('Exception 2: ', e2)

            print('Exception 1: ', e1)

            continue

        if N_days == 0:
    
            print('1회 실행으로 코드를 종료합니다.')
            print('============================================')
            break

        else:
       
            print('1시간 뒤 코드 재실행됩니다.')
            print('============================================')
            
            ts(3600)
        
    
    return df

In [3]:
df = Order_Automation(0)

DB 불러오기 완료!
--------------------------------------------
<Part 1. 취소 메일>
취소 건 확인을 시작합니다.
정상적으로 취소 메일을 불러왔습니다.
취소 건 확인 완료. 취소 건은 총 0 건입니다.
--------------------------------------------
<Part 2. 예약 메일>
예약 건 확인을 시작합니다.
정상적으로 예약 메일을 불러왔습니다.
주문번호 MZZ868997을 DB에 등록합니다
MZZ868997 - DB 등록 완료
주문번호 GWG619578을 DB에 등록합니다
GWG619578 - DB 등록 완료
주문번호 HAR717338을 DB에 등록합니다
HAR717338 - DB 등록 완료
모든 건이 예약이 완료되었습니다.
예약 건 확인 완료. 예약 건은 총 3 건입니다.
--------------------------------------------
소요 시간 : 73.52915906906128
완료 시간 : 2023-10-12 08:25:27.519114
--------------------------------------------
1회 실행으로 코드를 종료합니다.
